# Python 3.6 typing

Example taken from `typing` module documentation

http://mypy.readthedocs.io/en/latest/python2.html#multi-line-python-2-function-annotations


Add mypy type-checking cell magic to jupyter/ipython.

http://journalpanic.com/post/spice-up-thy-jupyter-notebooks-with-mypy/

https://gist.githubusercontent.com/knowsuchagency/f7b2203dd613756a45f816d6809f01a6/raw/c9c1b7ad9fa25a57ee1903d755d5525894ffa411/typecheck.py


In [1]:
!pip install mypy

In [2]:
"""
Add mypy type-checking cell magic to jupyter/ipython.

Save this script to your ipython profile's startup directory.

IPython's directories can be found via `ipython locate [profile]` to find the current ipython directory and ipython profile directory, respectively.

For example, this file could exist on a path like this on mac:

/Users/yourusername/.ipython/profile_default/startup/typecheck.py

where /Users/yourusername/.ipython/profile_default/ is the ipython directory for
the default profile.

The line magic is called "typecheck" to avoid namespace conflict with the mypy
package.

"""

from IPython.core.magic import register_cell_magic


@register_cell_magic
def typecheck(line, cell):
    """
    Run the following cell though mypy.

    Any parameters that would normally be passed to the mypy cli
    can be passed on the first line, with the exception of the
    -c flag we use to pass the code from the cell we want to execute

     i.e.

    %%typecheck --ignore-missing-imports
    ...
    ...
    ...

    mypy stdout and stderr will print prior to output of cell. If there are no conflicts,
    nothing will be printed by mypy.
    """

    from IPython import get_ipython
    from mypy import api

    # inserting a newline at the beginning of the cell
    # ensures mypy's output matches the the line
    # numbers in jupyter

    cell = '\n' + cell

    mypy_result = api.run(['-c', cell] + line.split())

    if mypy_result[0]:  # print mypy stdout
        print(mypy_result[0])

    if mypy_result[1]:  # print mypy stderr
        print(mypy_result[1])

    shell = get_ipython()
    shell.run_cell(cell)

In [3]:
%%typecheck --ignore-missing-imports

from sympy import sympify, init_printing
init_printing()

integer: int
    
integer = "mypy won't like this"

sympify('(e**3)**(1/2)')

"Notice the previous line was also printed out"


<string>:8: error: Incompatible types in assignment (expression has type "str", variable has type "int")



'Notice the previous line was also printed out'

In [4]:
%%typecheck --ignore-missing-imports

from typing import List

class Example:
    def method(self,
               lst:      List[str],
               opt:      int=0,
               *args:    str,
               **kwargs: bool) -> int:
        """Doc string comes after type comment."""
        
        print("Called `Example.method`...")
        
        return 0

    
ex = Example()

# Passes...
ex.method(['A','B','C'], 99, '123.456', flag=True)

# Fails...
ex.method([], 'A', 123, flag='False')

<string>:24: error: Argument 2 to "method" of "Example" has incompatible type "str"; expected "int"
<string>:24: error: Argument 3 to "method" of "Example" has incompatible type "int"; expected "str"
<string>:24: error: Argument 4 to "method" of "Example" has incompatible type "str"; expected "bool"

Called `Example.method`...
Called `Example.method`...
